In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

#  Define a tool with @tool decorator
@tool
def get_weather(location: str) -> str:
    """Return a mock weather forecast for a given location."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

# Initialize ToolNode with the tool
tool_node = ToolNode([get_weather])

# Bind tool to LLM model
model = init_chat_model(model="claude-3-5-haiku-latest")
model_with_tools = model.bind_tools([get_weather])

# Define state schema for messages
class MessagesState(TypedDict):
    messages: list

# Define agent node function
def call_model(state: MessagesState) -> MessagesState:
    response = model_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Function to check if model asked for tool usage
def should_continue(state: MessagesState):
    last = state["messages"][-1]
    if getattr(last, "tool_calls", None):
        return "tools"
    return END

# Build the state graph
builder = StateGraph(MessagesState)
builder.add_node("agent", call_model)
builder.add_node("tools", tool_node)
builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", should_continue, ["tools", END])
builder.add_edge("tools", "agent")

graph = builder.compile()

result = graph.invoke({"messages":[{"role":"user","content":"what's the weather in SF?"}]})
print(result)


ImportError: Unable to import langchain_anthropic. Please install with `pip install -U langchain-anthropic`